# Further VHDL and FPGA Design - Project

# Random Number Generation

## Contents

* [Seed Generation](#Seed-Generation)

* [Hardware Generated Random Numbers](#Harware-Generated-Random-Numbers)

* [Software Generated Random Numbers](#Software-Generated-Random-Numbers)

* [Applications of Random Numbers](#Applications-of-Random-Numbers)
    
    * [Password Generator](#Password-Generator)
    
    * [Cipher Generator](#Cipher-Generator)

## Setup

To setup the PYNQ board for this project, the following steps must be taken before running code:

* Connect the Grove Light Sensor to port G4 on the PMOD adaptor, and then plug the PMOD adaptor into the PMOD A port.
* Connect the Grove OLED Display to port G3 on the PMOD adaptor, and then plug the PMOD adaptor into the PMOD B port.
* Run the code below to import all the required libraries:

In [ ]:
#Libraries Needed
import random as r
import matplotlib.pyplot as plt
import time
import math
import string
from ipywidgets import *
from pynq import Overlay
from pynq.lib.pmod import Grove_Light
from pynq.lib.pmod import Grove_OLED
from pynq.lib.pmod import PMOD_GROVE_G3
from pynq.lib.pmod import PMOD_GROVE_G4 
from pynq.overlays.base import BaseOverlay
from IPython.display import display
base = BaseOverlay("base_new.bit")

## Seed Generation

When generating random numbers, a seed is required. This seed can be any number, but the seed determines the sequence of random numbers, so generating the seed in an unpredicatable fashion is crucial to secure random numbers. 

In this case, we use a light sensor connected to the PMOD A slot on the PYNQ board to generate a spread of values from the ambient light, and then pick one of the values at random, using the built in random function, which is seeded from the current time.

In [ ]:
lgt = Grove_Light(base.PMODA, PMOD_GROVE_G4)

ms_delay = 100
delay_s = 10
lgt.set_log_interval_ms(ms_delay)
lgt.start_log()
time.sleep(delay_s) # change input during this time
lgt_log = lgt.get_log()


limit = len(lgt_log)

index = r.randrange(0, len(lgt_log))
seedInt = lgt_log[index] * 100

seedInt = int(seedInt)

print('Seed is', seedInt)

The seed is then displayed on the OLED screen connected to PMOD B

In [ ]:
#connect to PMODB
pmod_oled = Grove_OLED(base.PMODB, PMOD_GROVE_G3)

pmod_oled.clear()
pmod_oled.write('Seed is ' + str(seedInt))

## Hardware Generated Random Numbers

A custom IP block was created in System Generator to generate random numbers using the programmable logic on the PYNQ board. The design for this is shown below in Figure 1:

![](./Uniform_32Bit/images/SimulinkModel.PNG)
<center>_Figure 1: Simulink Model_</center><br>

The ports to connect to the hardware design are then setup, as shown below:

In [ ]:
enable = 0x0C
load = 0x08
seed = 0x00
reset = 0x04
rand = 0x10
values = []

LFSR = base.uniform_0
LFSR.write(reset, 1)      
LFSR.write(reset, 0)                  # Unassert reset pin
LFSR.write(enable, 1)                 # Allow LFSR to operate
LFSR.write(load, 1)                   # Load seed int LFSR block
LFSR.write(seed, seedInt)   
LFSR.write(load, 0)    

The user is then prompted for the amount of numbers to be generated, and then the hardware design is used to generate the random numbers.

In [ ]:
totalNums_t = input('How many random numbers to be generated? \n')
totalNums = int(totalNums_t)

start = time.time()

for i in range(totalNums):               # Generate values
    values.append(LFSR.read(rand))    # append values to values array
    
timer_hw = time.time() - start

print(totalNums, 'random numbers generated. This took ', timer_hw, 'seconds.')

plt.hist(values)
plt.xlabel('Value')
plt.ylabel('Occurrence')
plt.title('Histogram')


## Software Generated Random Numbers

Using functions in the random library within Python, random numbers can also be generated using software. The code below shows the generation of the same amount of random numbers as the hardware generation, using the same seed provided by the external peripherals. 

In [ ]:
numbers = [] 
r.seed(seedInt)

start = time.time()

for i in range(totalNums):
    numbers.append(r.randint(0, 255))    
    
timer_sw = time.time() - start
print(totalNums, 'random numbers generated.\nThis took', timer_sw, 'seconds, compared to', timer_hw, 'seconds for hardware generation')

plt.hist(numbers)
plt.xlabel("Number")
plt.ylabel("Frequency")

## Applications of Random Numbers

### Password Generator

One possible application of random number generation is the generation of secure passwords, allowing secure passwords to be suggested when making new accounts for websites.

In [ ]:
pmod_oled = Grove_OLED(base.PMODB, PMOD_GROVE_G3)

Lowercase = True
Uppercase = True
Numbers = True
Symbols = True

def PassOptions(Lowercase, Uppercase, Numbers, Symbols):
    if Lowercase == True:
        a = string.ascii_lowercase
    if Lowercase == False:
        a = ''
    if Uppercase == True:
        b = string.ascii_uppercase
    if Uppercase == False:
        b = ''
    if Numbers == True:
        c = string.digits
    if Numbers == False:
        c = ''
    if Symbols == True:
        d = string.punctuation
    if Symbols == False:
        d = ''
    
    e = a+b+c+d
    return e
        

def gen(Length, Lowercase, Uppercase, Numbers, Symbols):
    password = [' '] * Length
    
    options = PassOptions(Lowercase, Uppercase, Numbers, Symbols)
    lim = len(options)
    
    for i in range(Length):
        password[i] = options[r.randrange(0, lim)]
    finalPassword = ''.join(password)
    print('Your Password Is: ',finalPassword)
    pmod_oled.clear()
    pmod_oled.write('Your Password Is\n '+ finalPassword)
    
    
    
Length=widgets.IntSlider(min=1,max=24,step=1,value=12)

interact(gen, Lowercase=True, Length=Length,  Uppercase=True, Numbers=True, Symbols=True);

### Cipher Generator

A cipher is a method of cryptography where the characters in a sentence are assigned a number (in this case their ASCII values), and then each number is shifted along by a set amount. This can be made more secure by using a set of randomly generated numbers, and using the same seed to encrypt and decrypt them.

In [ ]:
def cipher(message, key, mode):
    output = ""
    shift = []
    r.seed(key)
    for i in range(len(message)):
        shift.append(r.randrange(0, 123))
    
    if mode == 'Decrypt':
        for i in range(len(message)):
            letter = message[i]
            output += chr(ord(letter) - shift[i])
            
    if mode == 'Encrypt':
        for i in range(len(message)):
            letter = message[i]
            output += chr(ord(letter) + shift[i])
        
    print('\n', (mode+'ed'), 'Message: ',output)
    
message = widgets.Text(value='', placeholder='Enter Secret Message', description='Message:', disabled=False)
key = widgets.BoundedIntText(value=6, min=0, max=10000, step=1, description='Key', disabled=False)
mode = widgets.ToggleButtons( options=['Encrypt', 'Decrypt'], description='Select Mode:', disabled=False)
interact(cipher, message = message, key = key, mode = mode)